<a href="https://colab.research.google.com/github/NandanKumar07/Bone-Fracture-Classification-using-Deep-Learning/blob/main/Bone_Fracture_Identification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

cp: cannot stat '/content/drive/MyDrive/kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays/

Dataset URL: https://www.kaggle.com/datasets/vuppalaadithyasairam/bone-fracture-detection-using-xrays/versions/
License(s): unknown
 99% 171M/172M [00:00<00:00, 216MB/s]
100% 172M/172M [00:00<00:00, 195MB/s]


In [ ]:
# !unzip fracture-classification-dataset.zip
!unzip bone-fracture-detection-using-xrays.zip

Importing Libraries

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

Prepare Data:

In [ ]:
def load_and_preprocess_images(image_folder, target_size, grayscale=False):
    images = []
    for filename in os.listdir(image_folder):
        # Construct the full path to the image file
        img_path = os.path.join(image_folder, filename)
        # Load the image using OpenCV
        img = cv2.imread(img_path)
        # Resize the image to the target size
        img_resized = cv2.resize(img, target_size)
        # Convert the image to grayscale if grayscale=True
        if grayscale:
            img_resized = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
        # Append the preprocessed image to the list
        images.append(img_resized)
    # Convert the list of images to a NumPy array
    images_array = np.array(images)
    return images_array


In [ ]:
# Define the target size
target_size = (225, 225)

fractured_images_color = load_and_preprocess_images('boneData/train/fractured', target_size, grayscale=False)
non_fractured_images_color = load_and_preprocess_images('boneData/train/not fractured', target_size, grayscale=False)

# Concatenate fractured and non-fractured color images into a single array
all_images_color = np.concatenate([fractured_images_color, non_fractured_images_color], axis=0)

fractured_images_grayscale = load_and_preprocess_images('boneData/train/fractured', target_size, grayscale=True)
non_fractured_images_grayscale = load_and_preprocess_images('boneData/train/not fractured', target_size, grayscale=True)

# Concatenate fractured and non-fractured grayscale images into a single array
all_images_grayscale = np.concatenate([fractured_images_grayscale, non_fractured_images_grayscale], axis=0)

# Print the shapes of the resulting arrays
print("Shape of the array containing all color images:", all_images_color.shape)
print("Shape of the array containing all grayscale images:", all_images_grayscale.shape)

Shape of the array containing all color images: (8863, 225, 225, 3)
Shape of the array containing all grayscale images: (8863, 225, 225)


In [ ]:
# Normalize the grayscale images
all_img = all_images_grayscale / 255.0

In [ ]:
# Determine the number of fractured and non-fractured images
num_fractured_images = fractured_images_grayscale.shape[0]
num_non_fractured_images = non_fractured_images_grayscale.shape[0]

# Create binary labels
y_fractured = np.ones((num_fractured_images,), dtype=int)  # Label 1 for fractured images
y_non_fractured = np.zeros((num_non_fractured_images,), dtype=int)  # Label 0 for non-fractured images

# Concatenate binary labels
y = np.concatenate([y_fractured, y_non_fractured], axis=0)

# Print the shape of the labels array
print("Shape of the labels array:", y.shape)

Shape of the labels array: (8863,)


In [ ]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(all_images_grayscale, y, test_size = 0.20, random_state = 42)

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.25, random_state = 42)

# Print the shapes of the resulting arrays
print("Shape of x_train:", x_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of x_valid:", x_valid.shape)
print("Shape of y_valid:", y_valid.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of x_train: (5317, 225, 225)
Shape of y_train: (5317,)
Shape of x_valid: (1773, 225, 225)
Shape of y_valid: (1773,)
Shape of x_test: (1773, 225, 225)
Shape of y_test: (1773,)


In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation = 'relu', input_shape = (225, 225, 1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3, 3), activation = 'relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',  # Use the SGD optimizer
              loss='binary_crossentropy',  # Binary cross-entropy loss for binary classification
              metrics=['accuracy'])  # Monitor accuracy during training

# Train the model
history = model.fit(x_train.reshape(-1, 225, 225, 1), y_train, epochs=10, batch_size=64, validation_data=(x_valid.reshape(-1, 225, 225, 1), y_valid))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test.reshape(-1, 225, 225, 1), y_test)
print("Test Accuracy:", test_accuracy)

Epoch 1/10
84/84 [==============================] - 18s 108ms/step - loss: 11.2140 - accuracy: 0.6445 - val_loss: 0.5220 - val_accuracy: 0.7349
Epoch 2/10
84/84 [==============================] - 8s 90ms/step - loss: 0.3410 - accuracy: 0.8537 - val_loss: 0.3893 - val_accuracy: 0.8223
Epoch 3/10
84/84 [==============================] - 7s 89ms/step - loss: 0.1624 - accuracy: 0.9381 - val_loss: 0.2103 - val_accuracy: 0.9278
Epoch 4/10
84/84 [==============================] - 8s 90ms/step - loss: 0.0770 - accuracy: 0.9690 - val_loss: 0.2380 - val_accuracy: 0.9233
Epoch 5/10
84/84 [==============================] - 7s 88ms/step - loss: 0.0536 - accuracy: 0.9797 - val_loss: 0.1251 - val_accuracy: 0.9662
Epoch 6/10
84/84 [==============================] - 7s 89ms/step - loss: 0.0190 - accuracy: 0.9938 - val_loss: 0.1937 - val_accuracy: 0.9442
Epoch 7/10
84/84 [==============================] - 7s 88ms/step - loss: 0.0157 - accuracy: 0.9959 - val_loss: 0.1703 - val_accuracy: 0.9639
Epoch 8/10

In [ ]:
import cv2
import numpy as np

# Load and preprocess the image
image = cv2.imread('/content/IMG0002741.jpg')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
resized_image = cv2.resize(gray_image, (225, 225))    # Resize to the same dimensions used during training

# Make prediction
prediction = model.predict(np.expand_dims(resized_image, axis=0))

# Interpret the prediction
if prediction[0] >= 0.5:
    print("The bone contains a fracture.")
else:
    print("The bone does not contain a fracture.")


1/1 [==============================] - 0s 18ms/step
The bone contains a fracture.


In [ ]:
model.save('bone_fracture_model')

In [ ]:
model.save('/content/drive/My Drive/bone_fracture_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
